#### 问题1：请使用numpy来实现梯度下降并做一个单变量的线性回归。使用prob1.dat数据来验证你的算法。
#### 下面，我们将使用y = wX + b来表示该线性回归。


In [3]:
import os
from scipy import sparse
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  

'''
This problem is adapted from the course materials of IST 597 at Penn State University.
Thanks to the contribution of Alexander G. Ororbia II.
'''

# NOTE: 更新的步长、收敛标准和更新轮数
# meta-parameters for program
alpha = 0.01 # step size coefficient
eps = 1e-8 # controls convergence criterion
n_epoch = 1000 # number of epochs (full passes through the dataset)

In [12]:
def regress(X, theta):
    # 给定X和theta,输出y的预测值
    m = np.shape(X)[0]
    print("M=",m)
    input()
    b = theta[0]
    w = theta[1]
    y_hat = np.repeat(b[None,:], m, axis = 0) + np.dot(X, w.transpose())
    return y_hat

def computeCost(X,y,theta,reg):
    N = X.shape[0]
    y_mat = oneHotIt(y)
    probs = predict(X,theta)[1]
    loss = -np.sum(y_mat*np.log(probs))/N + (reg/2)*np.sum(theta[0]**2)   
    return loss

def computeCost(X, y, theta):
    # 计算当前的Rooted Mean Squared Error
    m = np.shape(y)[0]
    y_hat = regress(X, theta)
    loss = 1/(2*m)*np.sum(np.square(y_hat - y))
    return loss

def oneHotIt(y):
    N = y.shape[0]
    M = []
    for i in y:
        M.append([round(i[0])])
    M = np.mat(M)
    print(M)
    OHX = sparse.csr_matrix((np.ones(N), (M, np.array(range(N)))))
    OHX = np.array(OHX.todense()).T
    return OHX

def predict(X,theta):
    W = theta[0]
    b = theta[1]
    scores = np.dot(X,W)+b
    probs = np.apply_along_axis(softmax, 1, scores)
    return (scores, probs)

def computeGrad(X, y, theta): 
    dL_db = 0
    dL_dw = 0
    N = X.shape[0]
    y_mat = oneHotIt(y)
    probs = predict(X,theta)[1]
    ddy = (probs-y_mat)
    dL_dw = np.dot(X.T, ddy)/N + regress(X, theta)*theta[0] 
    dL_db = np.sum(ddy, axis=0)/N
    nabla = (dL_db, dL_dw)
    return nabla    

In [13]:
# 开始

path = '/storage/emulated/0/Kunologist/Jupyter/LectureNotes/ASSIGNMENT_2_FILES/prob1.dat'  
data = pd.read_csv(path, header=None, names=['X', 'Y']) 

data.describe()

# 转换数据
cols = data.shape[1]  
X = data.iloc[:,0:cols-1]  
y = data.iloc[:,cols-1:cols] 

X = np.array(X.values)  
y = np.array(y.values)

# 初始化线性回归的参数
w = np.zeros((1,X.shape[1]))
b = np.array([0])
theta = (b, w)

In [ ]:
# 第一次计算loss
L = computeCost(X, y, theta)
print("-1 L = {0}".format(L))
L_best = L
i = 0
cost = []
while(i < n_epoch):
    # TODO - 计算梯度，更新theta，计算loss，打印loss
    dL_dw, dL_db = computeGrad(X, y, theta)
    b = theta[1]
    w = theta[0]
    new_b = b - (alpha * dL_db)
    new_w = w - (alpha * dL_dw)
    theta = (new_w, new_b)
    L = computeCost(X, y, theta)
    print(" {0} L = {1}".format(i,L))
    i += 1
    
    cost.append(L)
    L_best = min(L_best, L)
    
# print parameter values found after the search
print("w = ",w)
print("b = ",b)

M= 97


In [ ]:
# 画出数据点
plt.scatter(X[:,0], y, s=30, label="Samples")
plt.xlabel("x")
plt.ylabel("y")
plt.legend(loc="best")
plt.savefig(os.path.join("prob1.png"))
plt.show()

In [ ]:
# 画出数据点和拟合的曲线
plt.plot(X_test, regress(X_test, theta), color = 'r', label="Model")
plt.scatter(X[:,0], y, s=30, label="Samples")
plt.xlabel("x")
plt.ylabel("y")
plt.xlim((np.amin(X_test) - kludge, np.amax(X_test) + kludge))
plt.ylim((np.amin(y) - kludge, np.amax(y) + kludge))
plt.legend(loc="best")
plt.savefig(str(alpha) + "_fit.png")
plt.show()

In [ ]:
# 画出loss收敛过程的曲线
plt.plot([i+1 for i in range(len(cost))], cost, label = "loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.savefig(str(alpha) + "_loss.png")
plt.show()